In [1]:
#a classic college level Monte Carlo simulation using real expected return and standard deviations for assets though sampling
#bond and stock retrns from a gaussian distribution

#An issue with this analysis is it assumes future returns will look like historical returns Ex. T-bonds have done very well over the timeframe of this analysis which can not continue due to hitting the effective lower bound

import numpy as np
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime

In [2]:
#read in data
start = datetime(2003,1,1)
end = datetime(2020,5,1)
TLT = data.DataReader("TLT", 'yahoo', start, end)
TLT_div = data.DataReader("TLT", 'yahoo-dividends', start, end)
IVV= data.DataReader("IVV", 'yahoo', start, end)
IVV_div= data.DataReader("IVV", 'yahoo-dividends', start, end)

In [3]:
#this portion is done and explained in Treasuries and IVV with dividends notebook
port=pd.concat([TLT["Close"],TLT_div["value"],IVV["Close"],IVV_div["value"]], axis=1,keys=["TLT","TLT_Div","IVV","IVV_Div"])
port["TLT_Div"]=port["TLT_Div"].fillna(0)
port["IVV_Div"]=port["IVV_Div"].fillna(0)
port["TLT_return"]=port["TLT"].pct_change()+port["TLT_Div"]/port["TLT"]
port["IVV_return"]=port["IVV"].pct_change()+port["IVV_Div"]/port["IVV"]
returns = port[["IVV_return", "TLT_return"]].copy()
An_port_returns = returns.resample('AS').sum().mean()
An_port_std = returns.resample('AS').sum().std()

In [4]:
#Problem set up: couple expecting to live 10 more years provides living expenses, portfolio composition
#and wants to know the probability of being able to make donation at end of life

#assumptions: annual rebalancing, withdraw living expenses anually at end of year

wealth =2e6
bnd_expt_return=An_port_returns["TLT_return"]       #TLT ave annual return
equity_expt_return=An_port_returns["IVV_return"]    #IVV ave annual return
bnd_std=An_port_std["TLT_return"]                   #TLT annual std
equity_std=An_port_std["IVV_return"]                #IVV annual std
equity_pct=.4                                       #percent of portfolio in equities rebalaced annually
withdraw=2e5                                        #annual withdrawal for living expence
donation=1e6                                        #wants to donate 1 mill at end of life

In [5]:
def wealth_planning(wealth,bnd_return,equity_expt_return,equity_std,equity_pct,withdraw):
    #year 1
    yr_start=wealth
    equity=yr_start*equity_pct
    bonds=yr_start*(1-equity_pct)
    equity_rtn=1+(equity_expt_return+np.random.normal(0,1)*equity_std)
    bond_rtn=1+(bnd_expt_return+np.random.normal(0,1)*bnd_std)
    yr_end=equity*equity_rtn+bonds*bond_rtn-withdraw
    
    for i in range(9):
        yr_start=yr_end
        equity=yr_start*equity_pct
        bonds=yr_start*(1-equity_pct)
        equity_rtn=1+(equity_expt_return+np.random.normal(0,1)*equity_std)
        bond_rtn=1+(bnd_expt_return+np.random.normal(0,1)*bnd_std)
        yr_end=equity*equity_rtn+bonds*bond_rtn-withdraw
        
    return(yr_end)

In [6]:
def simulation(num):
    sum=0
    sims=[]
    for i in range(num):
        sim=wealth_planning(wealth,bnd_expt_return,equity_expt_return,equity_std,equity_pct,withdraw)
        sims.append(sim)
        sum+=sim
    return(sims)

In [7]:
sims=pd.DataFrame(simulation(100000))

In [8]:
sims[sims>1e6].count()/1e5

0    0.7169
dtype: float64

In [9]:
sims.describe()

,0
count,1.000000e+05
mean,1.644372e+06
std,1.011878e+06
min,-9.622767e+05
25%,9.212616e+05
50%,1.494404e+06
75%,2.202351e+06
max,1.138687e+07
